In [1]:
%cd ..
%pwd

/workspace/DeepSpeech-2.x


'/workspace/DeepSpeech-2.x'

In [2]:
import argparse
import functools

from deepspeech.frontend.augmentor.augmentation import AugmentationPipeline
from deepspeech.frontend.featurizer.audio_featurizer import AudioFeaturizer
from deepspeech.frontend.normalizer import FeatureNormalizer
from deepspeech.utils.utility import add_arguments
from deepspeech.utils.utility import print_arguments

parser = argparse.ArgumentParser(description=__doc__)
add_arg = functools.partial(add_arguments, argparser=parser)
# yapf: disable
add_arg('num_samples',      int,    -1,    "# of samples to for statistics.")
add_arg('specgram_type',    str,
        'linear',
        "Audio feature type. Options: linear, mfcc, fbank.",
        choices=['linear', 'mfcc', 'fbank'])
add_arg('feat_dim',    int, 13, "Audio feature dim.")
add_arg('delta_delta',    bool,
        False,
        "Audio feature with delta delta.")
add_arg('stride_ms',    float, 10.0,  "stride length in ms.")
add_arg('window_ms',    float, 20.0,  "stride length in ms.")
add_arg('sample_rate',    int, 16000,  "target sample rate.")
add_arg('manifest_path',    str,
        'examples/aishell/s1/data/manifest.train.raw',
        "Filepath of manifest to compute normalizer's mean and stddev.")
add_arg('num_workers',
                        default=1,
                        type=int,
                        help='num of subprocess workers for processing')
add_arg('output_path',    str,
        'data/librispeech/mean_std.npz',
        "Filepath of write mean and stddev to (.npz).")
# yapf: disable
args = parser.parse_args([])
print(args)

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
[WARNING 2021/04/16 15:30:29 __init__.py:93] register user softmax to paddle, remove this when fixed!
2021-04-16 15:30:29,345 - WARNING - register user softmax to paddle, remove this when fixed!
[WARNING 2021/04/16 15:30:29 __init__.py:97] register user log_softmax to paddle, remove this when fixed!
2021-04-16 15:30:29,346 - WARNING - register user log_so

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/numba/core/types/__init__.py:108: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  long_ = _make_signed(np.long)
/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/numba/core/types/__init__.py:109: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by i

Namespace(delta_delta=False, feat_dim=13, manifest_path='examples/aishell/s1/data/manifest.train.raw', num_samples=-1, num_workers=1, output_path='data/librispeech/mean_std.npz', sample_rate=16000, specgram_type='linear', stride_ms=10.0, window_ms=20.0)


In [3]:
import random

import numpy as np
import paddle
from paddle.io import DataLoader
from paddle.io import Dataset

from deepspeech.frontend.audio import AudioSegment
from deepspeech.frontend.utility import load_cmvn
from deepspeech.frontend.utility import read_manifest

class CollateFunc(object):
    ''' Collate function for AudioDataset
    '''
    def __init__(self):
        pass
       
    def __call__(self, batch):
        mean_stat = None
        var_stat = None
        number = 0
        for feat in batch:
            sums = np.sum(feat, axis=1)
            if mean_stat is None:
                mean_stat = sums
            else:
                mean_stat += sums

            square_sums = np.sum(np.square(feat), axis=1)
            if var_stat is None:
                var_stat = square_sums
            else:
                var_stat += square_sums

            number += feat.shape[1]
        return paddle.to_tensor(number), paddle.to_tensor(mean_stat), paddle.to_tensor(var_stat)


class AudioDataset(Dataset):
    def __init__(self, manifest_path, feature_func, num_samples=-1, rng=None):
        self.feature_func = feature_func
        self._rng = rng
        manifest = read_manifest(manifest_path)
        if num_samples == -1:
            sampled_manifest = manifest
        else:
            sampled_manifest = self._rng.sample(manifest, num_samples)
        self.items = sampled_manifest

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        key = self.items[idx]['feat']
        audioseg = AudioSegment.from_file(key)
        feat = self.feature_func(audioseg)  #(D, T)
        return feat

/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:

augmentation_pipeline = AugmentationPipeline('{}')
audio_featurizer = AudioFeaturizer(
    specgram_type=args.specgram_type,
    feat_dim=args.feat_dim,
    delta_delta=args.delta_delta,
    stride_ms=args.stride_ms,
    window_ms=args.window_ms,
    n_fft=None,
    max_freq=None,
    target_sample_rate=args.sample_rate,
    use_dB_normalization=True,
    target_dB=-20)

def augment_and_featurize(audio_segment):
    augmentation_pipeline.transform_audio(audio_segment)
    return audio_featurizer.featurize(audio_segment)


collate_func = CollateFunc()

dataset = AudioDataset(
    args.manifest_path,
    augment_and_featurize, 
    args.num_samples)

batch_size = 20
data_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=args.num_workers,
    collate_fn=collate_func)

with paddle.no_grad():
    all_mean_stat = None
    all_var_stat = None
    all_number = 0
    wav_number = 0
    #     for i, batch in enumerate(data_loader()):
    for batch in data_loader():
        number, mean_stat, var_stat = batch
        if i == 0:
            all_mean_stat = mean_stat
            all_var_stat = var_stat
        else:
            all_mean_stat += mean_stat
            all_var_stat += var_stat
        all_number += number
        wav_number += batch_size

        if wav_number % 1000 == 0:
            print('process {} wavs,{} frames'.format(wav_number,
                                                           all_number))

cmvn_info = {
    'mean_stat': list(all_mean_stat.tolist()),
    'var_stat': list(all_var_stat.tolist()),
    'frame_num': all_number
}

Exception ignored in: <function _DataLoaderIterMultiProcess.__del__ at 0x7f9c9f91c2f0>
Traceback (most recent call last):
  File "/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 763, in __del__
    self._try_shutdown_all()
  File "/workspace/DeepSpeech-2.x/tools/venv/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 590, in _try_shutdown_all
    w.join()
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/usr/local/lib/python3.7/multiprocessing/popen_fork.py", line 48, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/local/lib/python3.7/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 
2021-04-16 15:44:43,413 - ERROR - DataLoader reader thread raised an exception!


SystemError: (Fatal) Blocking queue is killed because the data reader raises an exception.
  [Hint: Expected killed_ != true, but received killed_:1 == true:1.] (at /paddle/paddle/fluid/operators/reader/blocking_queue.h:158)
